# Simple tutorial of Atlas Search
We will enter a record and then start searching for it

In [ ]:
#Setup
import pymongo

ATLAS_URL = ''
DATABASE = 'atlas_tutorial'
COLLECTION = 'docs'

In [ ]:
client = pymongo.MongoClient(ATLAS_URL)
col = client[DATABASE][COLLECTION]

In [ ]:
# Delete all records from collection before we start the tutorial
col.delete_many({})

In [ ]:
col.insert_one({'text': 'This is a sample'})

In [ ]:
list(col.find({}))

In [ ]:
# Define some helper functions to generate queries
def build_query(search_term):
    return [
    { "$search":
     {"index": "default",
      "text":
      { "query": search_term,
        "path": {'wildcard': '*'}
      }
     }
    }
]

def search_query(search_term, show_score=False):
    query = build_query(search_term)
    if show_score:
        query.append({  "$set": { "score": { "$meta":  "searchScore" } } })
    results = col.aggregate(query)
    return list(results)

## Simple search query

In [ ]:
search_query('sample', show_score=True)
# We can see that it matches and finds the document. The score can be turned on and off.

In [ ]:
# This type of query will not match, as the word simple does not appear
search_query('simple', show_score=True)

## Fuzzy Matching
Instead of searching for `sample`, we'll incorrectly use the word `simple` and this time it will find a result.

In [ ]:
## Update our helper functions for fuzzy searching
def build_fuzzy_query(search_term):
    return [
    { "$search":
     {"index": "default",
      "text":
      { "query": search_term,
        "path": {'wildcard': '*'},
        "fuzzy": {"maxEdits": 2}
      }
     }
    }
]

def fuzzy_search_query(search_term, show_score=False):
    query = build_fuzzy_query(search_term)
    if show_score:
        query.append({  "$set": { "score": { "$meta":  "searchScore" } } })
    results = col.aggregate(query)
    return list(results)

In [ ]:
# Now we see that the document is found if we search for simple
fuzzy_search_query('simple')